In [1]:
!pip install boto3 mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.0/563.0 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━

In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("green-taxi-duration")

2024/10/04 08:55:43 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/1', creation_time=1728032143283, experiment_id='1', last_update_time=1728032143283, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
!wget -nc --directory-prefix data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet 
!wget -nc --directory-prefix data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet

--2024-10-04 08:55:43--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.111.148, 52.84.111.52, 52.84.111.169, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.111.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M  4.82MB/s    in 0.3s    

2024-10-04 08:55:44 (4.82 MB/s) - ‘data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2024-10-04 08:55:44--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.111.52, 52.84.111.169, 52.84.111.30, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.111.52|:443... connected.
HTTP reque

In [6]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [22]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs=-1)

    # X_train = dv.fit_transform(dict_train)
    # X_val = dv.transform(dict_val)
    # model.fit(X_train, y_train)
    
    pipeline = make_pipeline(dv, model)
    pipeline.fit(dict_train, y_train)

    # y_pred = model.predict(X_val)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    # With sklearn make_pipeline dictionary vectorizer artifact 
    # is in the pipeline object already.
    # with open("dict_vectorizer.bin", 'wb') as f_out:
    #     pickle.dump(dv, f_out)
    # mlflow.log_artifact("dict_vectorizer.bin")
    
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2024/10/04 09:16:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/04 09:16:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run gregarious-stoat-754 at: http://mlflow:5000/#/experiments/1/runs/59e900bae4a9494fa08e1017f98ae8fd.
2024/10/04 09:16:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/1.


In [16]:
from mlflow.tracking import MlflowClient


In [17]:
MLFLOW_TRACKING_URI = 'http://mlflow:5000'
RUN_ID = '3b9525d4d355455d9564a789109fa0de'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
# This doesnt work if the make_pipeline is used, because pipeline
# will do the dictionary vectorizing
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)
dv

DictVectorizer()

In [23]:
# tracking_uri is not required with S3 storage to download model
# mlflow server is not required for using the models
RUN_ID = "59e900bae4a9494fa08e1017f98ae8fd"
logged_model = f"s3://mlflow/1/{RUN_ID}/artifacts/model"
model = mlflow.pyfunc.load_model(logged_model)